## Connection VSCode to Google Colab
Article: https://www.programmersought.com/article/35047606912/

In the Colab Side

In [ ]:
import random, string, urllib.request, json, getpass

#Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

#Get public address and print connect command
with urllib.request.urlopen('http://0.0.0.0:4040/api/tunnels') as response:
    data = json.loads(response.read().decode())
    (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
    print(f'SSH command: ssh -p{port} root@{host}')

#Print root password
print(f'Root password: {password}')

# my_ngrok_private_token = '1s1nXCDfoNG9XQhawtGHWxbT7Ll_4sD6fUSPxn1Z7oCePXFkC'


In [ ]:
!service ssh status
!service ssh stop

In [ ]:
ON the VS Code side

In [1]:
#3000 is the port number of ngrok service;
#-auth is optional and provides basic authentication;
#-host-header allows access through the domain name defined in the local `hosts` file, in this example: `127.0.0.1 your-app.local`
ngrok http 3000 -auth="charles:123456" -host-header=unet.local
#8080 is the web service port number, use -region to specify the region: us, eu, ap, au, etc. The default is us
ngrok http 8080  

SyntaxError: invalid syntax (<ipython-input-1-6e3b23c547ca>, line 4)

In [ ]:
curl http://****.ngrok.io  #The url generated in the previous step, get
curl http://****.ngrok.io -d name=hxy #post